In [12]:
from pathlib import Path
import json
import numpy as np
import functools

In [3]:
def load_json(path: Path):
    with open(path, "r") as f:
        return json.load(f)
    

In [4]:
def pretty_print(array: np.ndarray):
    print(np.array2string(array, separator=", ", formatter={"float_kind": lambda x: "%.4f" % x}))

In [31]:
def split_transforms(path: Path, splits: int):
    transforms = load_json(path)
    frames = transforms["frames"]
    split_frames = np.array_split(frames, splits)
    
    split_indexes = []
    split_transforms = []
    for split in split_frames:
        split_transforms.append(
            {
                "camera_model": transforms["camera_model"],
                "fl_x": transforms["fl_x"],
                "fl_y": transforms["fl_y"],
                "cx": transforms["cx"],
                "cy": transforms["cy"],
                "w": transforms["w"],
                "h": transforms["h"],
                "k1": transforms["k1"],
                "k2": transforms["k2"],
                "p1": transforms["p1"],
                "p2": transforms["p2"],
                "frames": split.tolist(),
            }
        )
        if (len(split_indexes) == 0):
            split_indexes.append(len(split))
        else:
            split_indexes.append(split_indexes[-1] + len(split))
    return split_transforms, split_indexes


In [32]:
transforms_path = Path.cwd() / "baseline_transforms.json"
split_transforms, split_indexes = split_transforms(transforms_path, 4)

In [33]:
split_indexes

[275, 550, 824, 1098]

In [234]:
# Train the split transforms by themselves

In [235]:
# Retrieve the original, non-scaled and non-centered transforms


In [236]:
# Use the dataparser_transforms to scale and rotate the transforms
dataparser_transforms_path = Path.cwd() / "dataparser_transforms.json"
dataparser_transforms = load_json(dataparser_transforms_path)

dp_t = np.array(dataparser_transforms["transform"])
dp_scale = dataparser_transforms["scale"]

In [237]:
# Use the dp_t and dp_scale to scale and rotate the camera_to_world matrices in the camera_path file such that they're comparable to the split_transforms

transforms_matrix = np.array(split_transforms[0]["frames"][0]["transform_matrix"])
pretty_print(transforms_matrix)

[[-0.0000, -1.0000, -0.0000, -3.4461],
 [0.9848, -0.0000, 0.1736, 106.3866],
 [-0.1736, -0.0000, 0.9848, -2.3626],
 [0.0000, 0.0000, 0.0000, 1.0000]]


In [238]:
camera_path_path = Path.cwd() / "camera_path_one_lap.json"
camera_path = load_json(camera_path_path)
camera_path_c2w = np.array(camera_path["camera_path"][0]["camera_to_world"]).reshape(4,4,)
pretty_print(camera_path_c2w)

[[0.0387, 0.0485, 0.9981, 0.4484],
 [0.9993, -0.0019, -0.0387, 0.9853],
 [-0.0000, 0.9988, -0.0485, -0.0018],
 [0.0000, 0.0000, 0.0000, 1.0000]]


In [239]:
a = dp_t @ camera_path_c2w
a[:3, 3] *= dp_scale
a = np.vstack((a, [0, 0, 0, 1]))

pretty_print(a)

[[-0.0002, 0.9988, -0.0485, 0.2364],
 [0.9992, -0.0017, -0.0388, -0.3360],
 [-0.0389, -0.0485, -0.9981, -0.0437],
 [0.0000, 0.0000, 0.0000, 1.0000]]


In [240]:
# Get the distance between the two matrices trasform_matrix[:3, 3] and camera_to_world[:3, 3]
np.linalg.norm(transforms_matrix[:3, 3] - a[:3, 3])
# transforms_matrix[:3, 3]

106.81123024696991

In [241]:
b = [0.03871940596928383,0.9992501550311972,-2.220446121113367e-16,0,0.04846804299786681,-0.0018780620887794875,0.9988229486229716,0,0.9980739592677991,-0.03867383019325542,-0.04850441561635006,0,0.4483894695235361,0.985300560475246,-0.0018124304538501418,1]
b = np.array(b).reshape(4,4,)
pretty_print(b)

[[0.0387, 0.9993, -0.0000, 0.0000],
 [0.0485, -0.0019, 0.9988, 0.0000],
 [0.9981, -0.0387, -0.0485, 0.0000],
 [0.4484, 0.9853, -0.0018, 1.0000]]
